In [1]:
!pip list

Package             Version
------------------- ------------
asttokens           2.4.1
certifi             2024.8.30
charset-normalizer  3.4.0
colorama            0.4.6
comm                0.2.2
contourpy           1.3.0
cycler              0.12.1
debugpy             1.8.8
decorator           5.1.1
exceptiongroup      1.2.2
executing           2.1.0
filelock            3.13.1
fonttools           4.54.1
fsspec              2024.2.0
git-filter-repo     2.38.0
idna                3.10
importlib_metadata  8.5.0
importlib_resources 6.4.5
ipykernel           6.29.5
ipython             8.18.1
jedi                0.19.1
Jinja2              3.1.3
jupyter_client      8.6.3
jupyter_core        5.7.2
kiwisolver          1.4.7
MarkupSafe          2.1.5
matplotlib          3.9.2
matplotlib-inline   0.1.7
mpmath              1.3.0
nest-asyncio        1.6.0
networkx            3.2.1
numpy               1.26.3
opencv-python       4.10.0.84
optax               0.2.2
packaging           24.2
pandas      

In [2]:
import cv2
import os

image_dir = r"datasets\train\images"
label_dir = r"datasets\train\labels"
images = [img for img in os.listdir(image_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]
tracker = {}

for image_name in images:
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))
    
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f.readlines():
                label_data = line.strip().split()
                class_id = label_data[0]
                
                if(class_id in tracker.keys()):
                    tracker[class_id] += 1
                else:
                    tracker[class_id] = 1;

                x_center, y_center, box_width, box_height = map(float, label_data[1:])
                x_min = int((x_center - box_width / 2) * w)
                y_min = int((y_center - box_height / 2) * h)
                x_max = int((x_center + box_width / 2) * w)
                y_max = int((y_center + box_height / 2) * h)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
                cv2.putText(image, str(class_id), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    cv2.imshow("Image with Bounding Boxes", image)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

a = [int(i) for i in list(tracker.keys())]
a.sort()

for i in a:
    print(f"{i} -- {tracker[str(i)]}")

4 -- 1


In [3]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:12<00:00, 1.86MB/s]


In [4]:
results = model.train(
    data = r"C:\Users\Lenevo\#Achintya\College\Sem 5\French\Research\Code\datasets\data.yaml",
    imgsz = 640, 
    epochs = 100, 
    batch = 8, 
    name = "YOLOv8s_FSL_100_v1",
    device = 0)

New https://pypi.org/project/ultralytics/8.3.31 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.28  Python-3.9.20 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\Lenevo\#Achintya\College\Sem 5\French\Research\Code\datasets\data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=YOLOv8s_FSL_100_v1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnos

100%|██████████| 5.35M/5.35M [00:02<00:00, 2.26MB/s]


AMP: checks passed 


train: Scanning C:\Users\Lenevo\#Achintya\College\Sem 5\French\Research\Code\datasets\train\labels... 186 images, 0 backgrounds, 0 corrupt: 100%|██████████| 186/186 [00:00<00:00, 464.97it/s]

train: New cache created: C:\Users\Lenevo\#Achintya\College\Sem 5\French\Research\Code\datasets\train\labels.cache


...

    100/100      2.13G     0.3656     0.2368     0.9633          2        640: 100%|██████████| 24/24 [00:04<00:00,  4.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.92it/s]

                   all         18         18      0.981          1      0.995      0.928



100 epochs completed in 0.181 hours.
Optimizer stripped from runs\detect\YOLOv8s_FSL_100_v1\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\YOLOv8s_FSL_100_v1\weights\best.pt, 22.5MB

Validating runs\detect\YOLOv8s_FSL_100_v1\weights\best.pt...
Ultralytics 8.3.28  Python-3.9.20 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.47it/s]


                   all         18         18       0.98          1      0.995      0.925
                     b          4          4      0.974          1      0.995      0.936
                     j          2          2      0.965          1      0.995      0.927
                     n          3          3      0.981          1      0.995      0.846
                     o          2          2          1          1      0.995      0.895
                     r          5          5      0.975          1      0.995      0.995
                     u          2          2      0.984          1      0.995      0.951
Speed: 1.3ms preprocess, 12.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\YOLOv8s_FSL_100_v1
